# Sparse Method Algorithm for Research of Transcients Signal

Find transient source in data cubes

F. Acero, T. Stolarczyk, A. Chalumeau, Juin 2018

This notebook will:

    - Generate the fake data cubes (X,Y,T) (signal, background, total) for the Monte Carlo method
    - Filter the noise in the cubes using wavelet
    - Write the simulated datas on the disk
    - Measure how well the methods performed

All the code is pure Python (with the exception of one of the denoising method, which requires the binary file msvst_2d1d)

In [1]:
%matplotlib inline
import scipy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import PowerNorm
import matplotlib.gridspec as gridspec
#from matplotlib.colors import Normalize

from gammapy.stats import significance
from gammapy.stats import significance_on_off

from astropy.convolution import convolve,convolve_fft,Gaussian1DKernel,Gaussian2DKernel,Tophat2DKernel
from astropy.visualization import simple_norm
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord,Angle
from astropy.wcs import WCS
import astropy.units as u
from astropy.table import Table

from scipy.stats import norm
from scipy import interpolate
from scipy.ndimage import zoom

from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, fixed
import os,subprocess,sys,getpass,copy
import time as systime
import multiprocessing as mp
from functools import partial
from collections import defaultdict

#import line_profiler
#%load_ext line_profiler
#from scipy.stats import chisqprob
#from scipy.special import erfinv,erf,erfcx

import smartcube as smart
import scipy.integrate as sc
import pywi
from pywi.processing.transform import starlet

In [2]:
#### Global variables and initialisations
# Path to the binary file - This is lame and adding the binary file to somewhere accesible by $PATH shoud be enough.
#prog='/Users/facero/Documents/Work/CTA/CTA-AIM/code/image-cleaning/msvst/build/msvst_2d1d'

user = getpass.getuser()
print("Defining variables for user",user)
exit
if (user=="achalume"):
    root_dir   = "/Users/achalume/Documents/Stage/Codes/" 
    bin_dir    = "/Users/achalume/Documents/Lib_files/Sparse2D/build/"
elif (user=="stolar"):
    root_dir   = "/home/stolar/Code/" 
    bin_dir    = root_dir + "bin/"
elif (user=="vheyraud"):
    root_dir = ("./")
    bin_dir    = ""

#input_dir  = root_dir + "GRB-Fermi/FAWT3D/FermiData/"
#output_dir = root_dir + "GRB-Fermi/FAWT3D/output/"
#msvst      = bin_dir  + "msvst_2d1d"
#score_dir  = root_dir + "GRB-Fermi/FAWT3D/scores/"
#fig_dir    = root_dir + "GRB-Fermi/FAWT3D/Figures/"

input_dir  = root_dir
output_dir = "./bin/Monte_Carlo/"
msvst      = "msvst_2d1d"
score_dir  = root_dir
fig_dir    = root_dir
wave2D     = "./wl_t2_filter "
acceptance_file = "Acceptance-LST-ctapipe.txt"


#### Check existences
if not os.path.exists(output_dir) :
    os.makedirs(output_dir)
if not os.path.exists(fig_dir) :
    os.makedirs(fig_dir)
if not os.path.exists(score_dir) :
    os.makedirs(score_dir)
    
print("Directories :")
print("    - Root              :",root_dir)
print("    - MSVST_2d1d binary :",bin_dir)
print("    - Input             :",input_dir)
print("    - Output            :",output_dir)
print("    - Scores            :",score_dir)
print("    - Figures           :",fig_dir)

Defining variables for user vheyraud
Directories :
    - Root              : ./
    - MSVST_2d1d binary : 
    - Input             : ./
    - Output            : ./bin/Monte_Carlo/
    - Scores            : ./
    - Figures           : ./


# LC function

In [3]:
def LCBinChange(pre_t,pre_LC,fact_T_bin):
    """
    Re-bin a light curve multiplying the initial bin size with fact_T_bin. Returns : new_t,new_LC
    """
    
    alpha = fact_T_bin

    new_t = np.zeros(int(len(pre_t)/alpha)) ; new_LC = np.zeros(len(new_t))
    for i in range(0,len(new_t)):
        new_t[i] = pre_t[int(i*alpha)]
        new_LC[i] = np.sum(pre_LC[int((alpha*i)):int((alpha*i)+alpha)])

    return new_t,new_LC

############################################
def NormLC(LC,value,time,tmin,tmax):
    """
    Norm a range in a light curve (SUM(LC) = value from tmin to tmax). Returns : Normed_LC
    """
    LC_range = LC[np.where((time>=tmin) & (time<=tmax))]
    Normed_LC = value*LC/(np.sum(LC_range))
    
    return Normed_LC
                    
############################################
def InterpFunction(x_val,y_val,x_val_new):
    """
    Interpolate y(x) to get an approached function (y_new) with a given xtable (x_val_new). Returns : y_new
    """
    
    f = interpolate.interp1d(x_val,y_val)
    y_new = f(x_val_new)
    
    return y_new

############################################
def MakeLCTh(timebin,Xi,Tmintot,Tmaxtot,Ntotrange,Tminrange,Tmaxrange):
    """
    Make the theoretical LC normalizing at Ntotrange from Tminrange to Tmaxrange. Returns : Time, LC
    """
    
    T_0 = Tmintot
    Time = np.arange(Tmintot,Tmaxtot+timebin,timebin)
    K = (Tminrange**Xi)*(1+Xi)*Ntotrange/(Tmaxrange**(1+Xi) - Tminrange**(1+Xi))
    LC = K*Tminrange**(-Xi)*((Time)**(Xi+1)-(Time-timebin)**(Xi+1))/(Xi+1)
    
    return Time,LC
############################################
def MakeLCFa(datafile,Timebin,Bin_Factor,Ntotrange,Tminrange,Tmaxrange):
    """
    From Fabio's file, make LC normalizing at Ntotrange from Tminrange to Tmaxrange\
    choosing an initial bin size (Timebin). Returns : Time,LC
    """
    
    TimeData,LCData = np.loadtxt(datafile,unpack=True)
    Time = TimeData - TimeData[0] ; Time_size = TimeData[-1] - TimeData[0]   
    
    # Decrease time bin using an interpolation
    TimeInterp  = [x*Timebin for x in range(0,int(Time_size/Timebin),1)]
    TimeInterp = np.array(TimeInterp)
    LC = InterpFunction(Time,LCData,TimeInterp)
    
    LC_norm = NormLC(LC,Ntotrange,TimeInterp,Tminrange,Tmaxrange)
    T_input,LC_input = LCBinChange(TimeInterp,LC_norm,Bin_Factor) 
    
    return T_input,LC_input

############################################
def MakeLCIn(datafile,Timebin,Bin_Factor,Ntotrange,Tminrange,Tmaxrange):
    """
    From Inoue's extrapolated LC, make LC normalizing at Ntotrange from Tminrange to Tmaxrange.\
    Initial time bin (0.5) taken from the article. Returns : Time, LC
    """
    
    TData,LCData = np.genfromtxt(datafile,unpack=True,skip_header=6)

    # Set time values
    TData_0 = round(TData[0]) ; TData_end = np.round(TData[-1]/Timebin)*(Timebin)
    Time = np.linspace(TData_0,TData_end,len(TData))
    # Set LC values
    LC = LCData * Timebin # Put light curve in counts (it was in Cts/Bin before)
    LC = np.rint(LC).astype(int) # Set counts values

    T_input,LC_input = LCBinChange(Time,LC,Bin_Factor)
    LC_norm = NormLC(LC_input,Ntotrange,T_input,Tminrange,Tmaxrange)

    return T_input,LC_norm


# Cube fonctions

In [4]:
def BkgCube(time,SteadSrcPeak,bkg):
    """
    Creates the background model cube from a background value and a steady source peak.\
    Returns : BkgCube
    """
    BkgToycube = smart.CreateBkgModelCube(time,sigmaX,sigmaY,SteadSrcPeak,bkg,n2Dsize,pixelsize,Acceptance)
    BkgCube = smart.NoiseCube(BkgToycube)
    
    return BkgCube

############################################
def ImageCube(LC,time,SrcPeak,xcent,ycent,SteadSrcPeak,bkg):
    """
    Creates a cube containing bkg + transcient source. Returns : Fluctuated Signal+Bkg, Fluctuated Signal,Fluctuated Bkg
    """
    
    BkgToycube = smart.CreateBkgModelCube(time,sigmaX,sigmaY,SteadSrcPeak,bkg,n2Dsize,pixelsize,Acceptance)    
    
    #Create cubes' lists
    SrcCube = smart.CreateSignalModelCube(LC,SrcPeak,xcent,ycent,sigmaX,sigmaY,n2Dsize,pixelsize,Acceptance)
    SignalNoisyCube = smart.NoiseCube(SrcCube)
    BkgNoisyCube = smart.NoiseCube(BkgToycube)
    NoisyCube = SignalNoisyCube+BkgNoisyCube
    
    return (NoisyCube,SignalNoisyCube,BkgNoisyCube)

############################################
def BkgModelCube(time,tmin,tmax,tbin,SteadSrcPeak,bkg):
    """
    Create a bkg model cube (size = time) containing at each timebin the same image corresponding\
    to a perfectly known background. Returns : BkgModelCube
    
    Note : this function can be largely simplified, some parameters are useless (tmin,tmax,tbin..)
    """
        
    TimeAccum = np.arange(tmin,tmax+1,tbin)

    BkgToycube = smart.CreateBkgModelCube(TimeAccum,sigmaX,sigmaY,SteadSrcPeak,bkg,n2Dsize,pixelsize,Acceptance)
    
    #BkgNoisyCube = smart.NoiseCube(BkgToycube)
    BkgMeanImage = smart.CumulCube(BkgToycube)[:,:,-1]/(len(TimeAccum))
    
    BkgModelCube = np.zeros((len(BkgToycube[:,0,0]),len(BkgToycube[0,:,0]),len(time)))
    for t in range(0,len(time)):
        BkgModelCube[:,:,t] = BkgMeanImage
    
    return BkgModelCube

############################################
def BkgAcCube(tmin,tmax,tbin,SteadSrcPeak,bkg):
    """
    Create a bkg acquisition cube containing at each time bin the same image corresponding to
    and accumulated bkg from tmin to tmax(with tbins). Returns : BkgAcCue
    """
    TimeAccum = np.arange(tmin,tmax+1,tbin)
    
    BkgToycube = smart.CreateBkgModelCube(TimeAccum,sigmaX,sigmaY,SteadSrcPeak,bkg,n2Dsize,pixelsize,Acceptance)
    BkgNoisyCube = smart.NoiseCube(BkgToycube)

    return BkgNoisyCube

############################################
def WavCleaning(cube_noisy_file,wc_file,command):
    """
    Cube cleaning using wavelet transforms with msvst_2d1d (a shift is made to fix some argues,\
    maybe it should be seen better). Returns : Cube cleaned
    """
    
    cmd='%s  %s %s %s'%(msvst,command,cube_noisy_file,wc_file)
    subprocess.call(cmd,shell=True) # USE msvst_2d1d FILE

    #### Read wavelets cleaned cube file
    cube_wc,header_wc = smart.ReadCubeFromFile(wc_file,'fits')
    
    #### Cube shifting to fix msvst_2d1d's shift 
    cube_wc_shift = copy.deepcopy(cube_wc)
    cube_wc_shift[...,:-1] = cube_wc[...,1:] # Shift (time) to (time-1) (to see if its from msvst_2d1d)
    
    return cube_wc_shift

################################################
def WavCleaning2D(cube_noisy_file,wc_file,command):
    """
    Cube cleaning using wavelet transforms with mr_filter.
    Returns : Cube cleaned
    """
    
    cmd='%s  %s %s %s '%(wave2D,command,cube_noisy_file,wc_file)
    subprocess.call(cmd,shell=True)

    #### Read wavelets cleaned cube file
    cube_wc,header_wc = smart.ReadCubeFromFile(wc_file,'fits')
    
    return cube_wc
#################################################
def CreateAcceptance(flg,file):
    "flg = 0 : gaussian acceptance with width sigma_vig, flg = 1 interpolated acceptance from file"
    data = Table.read(filename=file,format="ascii")
    theta2 = data["theta2"]
    counts = data["counts"]
    Empirical = interpolate.interp1d(x=theta2,y=counts)
    x = np.linspace(0, n2Dsize-1, n2Dsize)
    y = np.linspace(0, n2Dsize-1, n2Dsize)
    x,y = np.meshgrid(x, y)
    Acceptance=np.empty((n2Dsize,n2Dsize))
    xo,yo=n2Dsize//2,n2Dsize//2
    
    if flg==0:
        Acceptance=np.exp( - ((x-xo)**2+(y-yo)**2)/(2*(sigma_vignetting/pixelsize)**2))
    if flg==1:
        theta2=((x-xo)**2+(y-yo)**2)*pixelsize**2
        theta2[xo,yo]=1# Necessary to avoid an error related to the interpolation error when theta2=0
        Acceptance=Empirical(theta2)
        Acceptance[xo,yo]=1# correct the value at the center
        #Acceptance=Empirical(((x-xo)**2+(y-yo)**2)*pixelsize**2+0.0002) : to use this we must add a small value cause the interpolation isn't defined in 0
    return Acceptance


# My Main

# Instrument response function

In [5]:
FOV               = 4.5    #width in degrees
pixelsize         = 0.1   #in degrees
n2Dsize           = int(FOV/pixelsize)
sigma_vignetting  = 1.8    # (deg) from ctobssim
ct95 = 0.2
a = 0.95
sigma_PSF         = smart.sigma_from_containment_radius(ct95,a)   # (deg) Since the psf isn't a gaussian, we use half of the 95% containement radius and the shape of a gaussian
background        = 0.04    # cts/s/pixel in E=50-500 GeV at center of FoV. Tbin=10s bkg=0.1
sigmaX, sigmaY   = float(sigma_PSF/pixelsize),  float(sigma_PSF/pixelsize) # Point like source
Acceptance=CreateAcceptance(0,acceptance_file)

print("Instrument response function")
print("----------------------------")
print("  Field of view       : ",FOV)
print("  Pixel size          : ",pixelsize)
print("  N. of pixels        : ",n2Dsize,"x",n2Dsize)
print("  Mean CTA background : ",background," (cts/s/pix, 50<E<500 GeV)")



Instrument response function
----------------------------
  Field of view       :  4.5
  Pixel size          :  0.1
  N. of pixels        :  45 x 45
  Mean CTA background :  0.04  (cts/s/pix, 50<E<500 GeV)


# Make LC and cubes

In [6]:
##################################
# Create 3 LC choosing normalisation range (t_min_norm,t_max_norm) and value (Ntot_norm)
# Each LC and time correspond to a name.
# Timebin[name] = Tbin_input[name]
# Time,LC_Values = T_input[name],LC_input[name]
##################################

LC_input,T_input,t_min_norm,t_max_norm = {},{},{},{} ; Tbin_input = {}

#### Make theoretical LC
Th_name = 'Theoretical'
Tbin_input[Th_name] = 4 # se
xi   = -1.4
t_min_Th = 30 ; t_0_Th = t_min_Th
t_max_Th = 200
Ntot_norm_Th = 1115 # From Inoue 
t_min_norm_Th = 50
t_max_norm_Th = 94
T_input[Th_name],LC_input[Th_name] =\
    MakeLCTh(Tbin_input[Th_name],xi,t_min_Th,t_max_Th,Ntot_norm_Th,t_min_norm_Th,t_max_norm_Th)

#### Make LC from Fabio
Fa_name = 'Fabio LC'
TBin_Fa = 1
Bin_Factor_Fa = 6
Tbin_input[Fa_name] = TBin_Fa*Bin_Factor_Fa
Ntot_norm_Fa = 200
t_min_norm_Fa = 90
t_max_norm_Fa = 110
data_file_Fabio = input_dir+"LAT-GRB130427.dat"
T_input[Fa_name],LC_input[Fa_name] =\
   MakeLCFa(data_file_Fabio,TBin_Fa,Bin_Factor_Fa,Ntot_norm_Fa,t_min_norm_Fa,t_max_norm_Fa)

#### Make LC from Inoue
In_name = 'Inoue LC'
TBin_data_In = 0.5
Bin_Factor_In = 8
Tbin_input[In_name] = TBin_data_In*Bin_Factor_In
Ntot_norm_In = 1115
t_min_norm_In = 50
t_max_norm_In = 94
data_file_Inoue = input_dir+"Inoue_LC_080916C.dat"
T_input[In_name],LC_input[In_name] =\
    MakeLCIn(data_file_Inoue,TBin_data_In,Bin_Factor_In,Ntot_norm_In,t_min_norm_In,t_max_norm_In)

##################################
# Choose main parameters : Source position and values (list), steady source value, which LC from the 3.
# Then select the differents timeranges from tnew_min until Tnew_max (array)
# Get a dict containing a LC with different sizes
##################################

xcenter, ycenter = 22,22#25,25#10,10# Position Center of GRB on Image (px,px)
Relative_effective_area_correction=1#0.73 #1#0.9 for a 1.4° offset, 0.73 for a 2.47° offset 
SteadySourcePeak = 0. # Quiescent source - intensity

name = Th_name # Th_name ; Fa_name ; In_name
TimeInput = T_input[name]
LCInput = LC_input[name]*Relative_effective_area_correction
Bkg_input = Tbin_input[name]*background

Tnew_min = 24 
Tmax=174

T_new,LC_new = {},{}
T_new = TimeInput[np.where((TimeInput>=(TimeInput[0]+Tnew_min)) & (TimeInput<=Tmax))]
LC_new = LCInput[np.where((TimeInput>=(TimeInput[0]+Tnew_min)) & (TimeInput<=Tmax))]
print(T_new)

#print('tmin = %ss, tmax values (s) = %s\nTimebin = %s s'%(np.min(T_new[tmax]),Tnew_max,Tbin_input[name]))

[ 54  58  62  66  70  74  78  82  86  90  94  98 102 106 110 114 118 122
 126 130 134 138 142 146 150 154 158 162 166 170 174]


# Performances : Monte-Carlo

In [7]:
## Set source intensity list

RedFactorList = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,np.inf]#

print(RedFactorList)

[1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, inf]


In [8]:
%%time
## Monte-Carlo simulation

Niterate = 400
Tmax=T_new[-1]
wav = 2 # choose the wavelet filtering method : 1 : 2D1D, 2 : 2D PyWI, 3 : 1+2, 4 : 2D Sparse2D

# Known background methods
TminBkg, TmaxBkg,TbinBkg = 0,60,Tbin_input[name]

# Wavelets 2D1D method
cmd2D1D = '-N4 -f3 -n4 -F2 -s5 -T'

#cmd = '-f2 -F2 -s2  -T'
#Wavelets 2D method
cmd2D = '-n4 -m2 -P -s3 -k -K -C1 -F2 -v'

# Wavelets 2DPywi method
threshold_list=[100,0]#


for i in range(0,Niterate):
    
    ## Create the fluctuated background model and the acquired background
    
    BkgModel = BkgModelCube(T_new,TminBkg,TmaxBkg,TbinBkg,SteadySourcePeak,Bkg_input)
    BkgAc = BkgAcCube(TminBkg,TmaxBkg,TbinBkg,SteadySourcePeak,Bkg_input)
        
    smart.WriteCubeToFile(BkgAc,output_dir+'AcBkgCube_%i_%i.fits'%(TmaxBkg-TminBkg,i),'')
    smart.WriteCubeToFile(BkgModel,output_dir+'ModelBkg_%i_%i.fits'%(Tmax,i),'')

for RedFactor in RedFactorList:
    
    for i in range(0,Niterate):


        ## Create the simulated signal, background and signal+background, the model background and accumulated background cubes
        
        CubeList = ImageCube(LC_new,T_new,1/RedFactor,xcenter,ycenter,SteadySourcePeak,Bkg_input)
        Total = CubeList[0]
        Signal = CubeList[1]
        Bkg = CubeList[2]
        
        WaveFilt = np.zeros(np.shape(Total))
        
        ## Write the created cubes in specific directories in the bin

        smart.WriteCubeToFile(Signal,output_dir+'FlucSignal_%s_%i.fits'%(RedFactor,i),'')
        smart.WriteCubeToFile(Total,output_dir+'FlucTotal_%s_%i.fits'%(RedFactor,i),'')
        smart.WriteCubeToFile(Bkg,output_dir+'FlucBkg_%s_%i.fits'%(RedFactor,i),'')

        
        # WAVELETS
        
        if wav==1 or wav==3 :
            
            for t in T_new:
            
                Cubefile = output_dir+'FlucTotal_%s_%i_%i.fits'%(RedFactor,i,t)
                WCubefile = output_dir+'WavCube2D1D_%s_%i_%i.fits'%(RedFactor,i,t)
                
                k=np.int(np.where(T_new==t)[0])

                Cube_temp = Total[:,:,:k+1]
                smart.WriteCubeToFile(Cube_temp,Cubefile,'')
                
                WavCleaning(Cubefile,WCubefile,cmd2D1D) 
                
        if wav==2 or wav==3 :
            
            Total=smart.ReadCubeFromFile(output_dir+'FlucTotal_%s_%i.fits'%(RedFactor,i),"fits")[0]#to be removed
            
            for k in range(len(T_new)):
                
                img=smart.CumulCube(Total)[:,:,k]
                WaveFilt[:,:,k]=pywi.processing.compositing.filter_with_starlet.clean_image(img,type_of_filtering='ksigma_hard_filtering',\
                            filter_thresholds=threshold_list,last_scale_treatment='mask',detect_only_positive_structures=True)
                smart.WriteCubeToFile(WaveFilt,output_dir+'WavCube2D_%s_%i.fits'%(RedFactor,i),'')
        
        if wav==4 :
            
            for t in T_new:
                
                Cubefile = output_dir+'FlucTotal_%s_%i_%i.fits'%(RedFactor,i,t)
                WCubefile = output_dir+'WavCube_%s_%i_%i.fits'%(RedFactor,i,t)
                
                k=np.int(np.where(T_new==t)[0])

                Cube_temp = Total[:,:,:k+1]
                smart.WriteCubeToFile(np.nansum(Cube_temp,axis=2),Cubefile,'')
                
                temp=WavCleaning2D(Cubefile,WCubefile,cmd2D) 
            
        print(i+1,'done in',Niterate,end='\r')


CPU times: user 27min 59s, sys: 2min 36s, total: 30min 36s
Wall time: 30min 32s
